# Chapter 9: Classes and Objects

## Introduction

SystemVerilog introduces object-oriented programming (OOP) concepts to hardware description and verification. Classes provide a powerful way to create reusable, modular code structures that can model complex data types and behaviors. This chapter covers the fundamental concepts of classes and objects in SystemVerilog.

## Class Declarations

A class in SystemVerilog is a user-defined data type that encapsulates data (properties) and functions (methods) that operate on that data. Classes serve as templates for creating objects.

### Basic Class Syntax

```systemverilog
class ClassName;
    // Properties (data members)
    // Methods (functions and tasks)
endclass
```

### Simple Class Example

```systemverilog
class Packet;
    // Properties
    bit [7:0] header;
    bit [31:0] payload;
    bit [7:0] checksum;
    
    // Method to display packet contents
    function void display();
        $display("Header: %h, Payload: %h, Checksum: %h", 
                 header, payload, checksum);
    endfunction
endclass
```

### Class with Constructor

```systemverilog
class Transaction;
    rand bit [31:0] addr;
    rand bit [31:0] data;
    bit [1:0] cmd;
    
    // Constructor
    function new(bit [1:0] command = 0);
        cmd = command;
        // Randomize other fields
        assert(randomize());
    endfunction
    
    // Method to check transaction validity
    function bit is_valid();
        return (addr != 0 && cmd != 2'b11);
    endfunction
endclass
```

### **Example 1: Basic Class Template**
Simple class structure showing the fundamental syntax with properties and a basic method.

In [6]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_1__basic_class_template/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// basic_class_example_testbench.sv
import counter_pkg::*;                      // Import all items from package

module basic_class_test_bench;              // Testbench module
  simple_counter_class my_counter;          // Class handle (object reference)
  simple_counter_class second_counter;      // Another class handle
  
  initial begin
    // Dump waves for simulation
    $dumpfile("basic_class_test_bench.vcd");
    $dumpvars(0, basic_class_test_bench);
    
    $display("=== Basic Class Template Example (Package Version) ===");
    $display();
    
    // Create first counter object
    my_counter = new("primary_counter");    // Instantiate class object
    
    // Test basic operations on first counter
    my_counter.increment();                 // Call increment method
    my_counter.increment();                 // Increment again
    my_counter.increment();                 // One more time
    
    $display("Current count: %0d", my_counter.get_count());
    $display();
    
    // Create second counter object with different name
    second_counter = new("backup_counter"); // Another object instance
    
    // Test operations on second counter
    second_counter.increment();             // This counter is independent
    $display("Second counter: %0d", second_counter.get_count());
    $display("First counter still: %0d", my_counter.get_count());
    $display();
    
    // Test reset functionality
    my_counter.reset();                     // Reset first counter
    $display("After reset: %0d", my_counter.get_count());
    
    $display();
    $display("=== Package-Based Class Example Complete ===");
    
    #1;                                     // Wait for a time unit
  end

endmodule
```

```systemverilog
// counter_package.sv
package counter_pkg;                        // SystemVerilog package definition

  class simple_counter_class;               // Basic class definition
    int count_value;                        // Class property (data member)
    string counter_name;                    // Another class property
    
    // Constructor method - called when object is created
    function new(string name = "default_counter");
      counter_name = name;                  // Initialize counter name
      count_value = 0;                      // Initialize count to zero
      $display("Created counter: %s", counter_name);
    endfunction
    
    // Method to increment the counter
    function void increment();
      count_value++;                        // Increment the count
      $display("%s: count = %0d", counter_name, count_value);
    endfunction
    
    // Method to get current count value
    function int get_count();
      return count_value;                   // Return current count
    endfunction
    
    // Method to reset the counter
    function void reset();
      count_value = 0;                      // Reset count to zero
      $display("%s: reset to 0", counter_name);
    endfunction

  endclass

endpackage : counter_pkg                    // End package with explicit name
```

Verilator Simulation Output:
=== Basic Class Template Example (Package Version) ===

Created counter: primary_counter
primary_counter: count = 1
primary_counter: count = 2
primary_counter: count = 3
Current count: 3

Created counter: backup_counter
backup_counter: count = 1
Second counter: 1
First counter still: 3

primary_counter: reset to 0
After reset: 0

Process finished with return code: 0
Removing Chapter_9_examples/example_1__basic_class_template/obj_dir directory...
Chapter_9_examples/example_1__basic_class_template/obj_dir removed successfully.


0

### **Example 2: Simple Packet Class**
Basic class representing a network packet with header, payload, and display functionality.

In [18]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_2__simple_packet_class/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// simple_packet_class.sv
class NetworkPacket;
  // Class properties
  rand bit [15:0] header_length;    // Header length in bytes
  rand bit [31:0] packet_id;        // Unique packet identifier
  rand bit [7:0]  packet_type;      // Type of packet (TCP, UDP, etc.)
  rand bit [15:0] payload_size;     // Payload size in bytes
  rand bit [7:0]  payload_data[];   // Dynamic array for payload
  
  // Constructor
  function new(bit [15:0] hdr_len = 20, bit [15:0] payload_sz = 64);
    // Set fixed values first
    this.header_length = hdr_len;
    this.payload_size = payload_sz;
    
    // Create payload array with correct size
    this.payload_data = new[int'(payload_sz)];
    
    // Set random values for ID and type
    this.packet_id = $urandom();
    this.packet_type = 8'($urandom_range(0, 255));
    
    // Fill payload with pattern
    foreach(payload_data[i]) begin
      payload_data[i] = 8'(i % 256);
    end
  endfunction
  
  // Method to display packet information
  function void display_packet_info();
    $display("=== Network Packet Information ===");
    $display("Header Length: %0d bytes", header_length);
    $display("Packet ID:     0x%08h", packet_id);
    $display("Packet Type:   0x%02h", packet_type);
    $display("Payload Size:  %0d bytes", payload_size);
    $display("Total Size:    %0d bytes", header_length + payload_size);
  endfunction
  
  // Method to display first few bytes of payload
  function void display_payload_sample(int num_bytes = 8);
    $display("--- Payload Sample (first %0d bytes) ---", num_bytes);
    $write("Payload: ");
    for(int i = 0; i < num_bytes && i < payload_data.size(); i++) begin
      $write("0x%02h ", payload_data[i]);
    end
    $display();
  endfunction
  
  // Method to calculate total packet size
  function int get_total_size();
    return int'(header_length) + int'(payload_size);
  endfunction
  
  // Method to check if packet is valid
  function bit is_valid_packet();
    return (header_length > 0) && (payload_size > 0) && 
           (int'(payload_data.size()) == int'(payload_size));
  endfunction
  
endclass : NetworkPacket

// Design module that demonstrates packet usage
module packet_processor;
  NetworkPacket pkt;
  
  initial begin
    $display();
    $display("=== Packet Processor Module ===");
    
    // Create a packet instance
    pkt = new(24, 128);  // 24-byte header, 128-byte payload
    
    // Display packet information
    pkt.display_packet_info();
    pkt.display_payload_sample(12);
    
    // Validate packet
    if(pkt.is_valid_packet()) begin
      $display("Packet validation: PASSED");
    end else begin
      $display("Packet validation: FAILED");
    end
    
    $display("Total packet size: %0d bytes", pkt.get_total_size());
    $display();
  end
  
endmodule : packet_processor
```

```systemverilog
// simple_packet_class_testbench.sv
module packet_testbench;
  // Instantiate the packet processor design
  packet_processor PACKET_PROC_INSTANCE();
  
  // Local packet instances for testing
  NetworkPacket test_packet_1;
  NetworkPacket test_packet_2;
  NetworkPacket test_packet_3;
  
  initial begin
    // Setup for waveform dumping
    $dumpfile("packet_testbench.vcd");
    $dumpvars(0, packet_testbench);
    
    $display();
    $display("Testbench starting...");
    $display();
    
    // Wait for design module to complete
    #1;
    
    // Test 1: Create small packet
    $display("=== Test 1: Small Packet ===");
    test_packet_1 = new(16, 32);  // Small packet: 16-byte header, 32-byte payload
    test_packet_1.display_packet_info();
    test_packet_1.display_payload_sample(6);
    $display("Is valid: %s", test_packet_1.is_valid_packet() ? "YES" : "NO");
    $display();
    
    // Test 2: Create large packet
    $display("=== Test 2: Large Packet ===");
    test_packet_2 = new(40, 1024); // Large packet: 40-byte header, 1KB payload
    test_packet_2.display_packet_info();
    test_packet_2.display_payload_sample(10);
    $display("Is valid: %s", test_packet_2.is_valid_packet() ? "YES" : "NO");
    $display();
    
    // Test 3: Create standard packet and randomize
    $display("=== Test 3: Randomized Packet ===");
    test_packet_3 = new();  // Use default constructor
    // Re-randomize only packet_id and packet_type
    test_packet_3.packet_id = $urandom();
    test_packet_3.packet_type = 8'($urandom_range(0, 255));
    test_packet_3.display_packet_info();
    test_packet_3.display_payload_sample(8);
    $display("Is valid: %s", test_packet_3.is_valid_packet() ? "YES" : "NO");
    $display();
    
    // Test 4: Compare packet sizes
    $display("=== Test 4: Packet Size Comparison ===");
    $display("Packet 1 total size: %0d bytes", test_packet_1.get_total_size());
    $display("Packet 2 total size: %0d bytes", test_packet_2.get_total_size());
    $display("Packet 3 total size: %0d bytes", test_packet_3.get_total_size());
    
    if(test_packet_2.get_total_size() > test_packet_1.get_total_size()) begin
      $display("Packet 2 is larger than Packet 1 (as expected)");
    end
    $display();
    
    // Final summary
    $display("=== Testbench Summary ===");
    $display("Created and tested 3 different packet instances");
    $display("Demonstrated class instantiation and method calls");
    $display("Verified packet validation and size calculation");
    $display("Showed dynamic array usage in payload");
    $display();
    
    $display("Hello from packet class testbench!");
    $display();
    
    // Finish simulation
    $finish;
  end
  
endmodule : packet_testbench
```

Verilator Simulation Output:

=== Packet Processor Module ===
=== Network Packet Information ===
Header Length: 24 bytes
Packet ID:     0x797673c4
Packet Type:   0x9c
Payload Size:  128 bytes
Total Size:    152 bytes
--- Payload Sample (first 12 bytes) ---
Payload: 0x00 0x01 0x02 0x03 0x04 0x05 0x06 0x07 0x08 0x09 0x0a 0x0b
Packet validation: PASSED
Total packet size: 152 bytes


Testbench starting...

=== Test 1: Small Packet ===
=== Network Packet Information ===
Header Length: 16 bytes
Packet ID:     0x9efdd502
Packet Type:   0xe4
Payload Size:  32 bytes
Total Size:    48 bytes
--- Payload Sample (first 6 bytes) ---
Payload: 0x00 0x01 0x02 0x03 0x04 0x05
Is valid: YES

=== Test 2: Large Packet ===
=== Network Packet Information ===
Header Length: 40 bytes
Packet ID:     0x92178378
Packet Type:   0xbc
Payload Size:  1024 bytes
Total Size:    1064 bytes
--- Payload Sample (first 10 bytes) ---
Payload: 0x00 0x01 0x02 0x03 0x04 0x05 0x06 0x07 0x08 0x09
Is valid: YES

=== Test 3: Randomi

0

### **Example 3: Transaction with Constructor**
Class demonstrating constructor usage with default parameters and basic validation.

In [28]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_3__transaction_constructor/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// packet_transaction.sv
class packet_transaction;
  // Transaction properties
  rand bit [7:0]  src_addr;
  rand bit [7:0]  dst_addr; 
  rand bit [15:0] payload_size;
  rand bit [1:0]  pkt_priority;
  bit             valid;
  
  // Constructor with default parameters and validation
  function new(bit [7:0] src = 8'h00, 
               bit [7:0] dst = 8'hFF,
               bit [15:0] size = 16'd64,
               bit [1:0] prio = 2'b01);
    
    $display("Creating new packet transaction...");
    
    // Assign parameters to class properties
    src_addr = src;
    dst_addr = dst;
    payload_size = size;
    pkt_priority = prio;
    
    // Basic validation
    if (validate_transaction()) begin
      valid = 1'b1;
      $display("Transaction validation: PASSED");
    end else begin
      valid = 1'b0;
      $display("Transaction validation: FAILED");
    end
    
    $display("Constructor completed");
    $display();
  endfunction
  
  // Validation function
  function bit validate_transaction();
    bit validation_result = 1'b1;
    
    // Check if source and destination are different
    if (src_addr == dst_addr) begin
      $display("ERROR: Source and destination addresses are the same!");
      validation_result = 1'b0;
    end
    
    // Check payload size limits (minimum 1, maximum 1024)
    if (payload_size == 0 || payload_size > 1024) begin
      $display("ERROR: Invalid payload size: %0d", payload_size);
      validation_result = 1'b0;
    end
    
    return validation_result;
  endfunction
  
  // Display transaction details
  function void display_transaction();
    $display("=== Packet Transaction Details ===");
    $display("Source Address:  0x%02h", src_addr);
    $display("Dest Address:    0x%02h", dst_addr);
    $display("Payload Size:    %0d bytes", payload_size);
    $display("Priority:        %0d", pkt_priority);
    $display("Valid:           %0s", valid ? "YES" : "NO");
    $display("===================================");
    $display();
  endfunction
  
endclass
```

```systemverilog
// packet_transaction_testbench.sv
module packet_constructor_testbench;
  
  // Transaction handles
  packet_transaction default_packet;
  packet_transaction custom_packet;
  packet_transaction invalid_packet;
  packet_transaction partial_packet;
  
  initial begin
    // Dump waves for visualization
    $dumpfile("packet_constructor_testbench.vcd");
    $dumpvars(0, packet_constructor_testbench);
    
    $display();
    
    // Test 1: Create packet with default constructor parameters
    $display("Test 1: Creating packet with default parameters");
    $display("--------------------------------------------------------");
    default_packet = new();  // Uses all default values
    default_packet.display_transaction();
    
    // Test 2: Create packet with custom parameters
    $display("Test 2: Creating packet with custom parameters");
    $display("--------------------------------------------------------");
    custom_packet = new(8'hA0, 8'hB5, 16'd256, 2'b11);
    custom_packet.display_transaction();
    
    // Test 3: Create invalid packet (same src/dst) to test validation
    $display("Test 3: Creating invalid packet (same source/destination)");
    $display("--------------------------------------------------------");
    invalid_packet = new(8'hCC, 8'hCC, 16'd128, 2'b10);  // Same src/dst
    invalid_packet.display_transaction();
    
    // Test 4: Demonstrate partial parameter usage
    $display("Test 4: Creating packet with partial custom parameters");
    $display("--------------------------------------------------------");
    partial_packet = new(8'h11, 8'hFF, 16'd512, 2'b01);  // Custom src & size
    partial_packet.display_transaction();
    
    $display("=== Constructor Demo Completed ===");
    $display();
    
    #10;  // Small delay before ending
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:

Test 1: Creating packet with default parameters
--------------------------------------------------------
Creating new packet transaction...
Transaction validation: PASSED
Constructor completed

=== Packet Transaction Details ===
Source Address:  0x00
Dest Address:    0xff
Payload Size:    64 bytes
Priority:        1
Valid:           YES

Test 2: Creating packet with custom parameters
--------------------------------------------------------
Creating new packet transaction...
Transaction validation: PASSED
Constructor completed

=== Packet Transaction Details ===
Source Address:  0xa0
Dest Address:    0xb5
Payload Size:    256 bytes
Priority:        3
Valid:           YES

Test 3: Creating invalid packet (same source/destination)
--------------------------------------------------------
Creating new packet transaction...
ERROR: Source and destination addresses are the same!
Transaction validation: FAILED
Constructor completed

=== Packet Transaction Details =

0

## Properties and Methods

Properties are the data members of a class, while methods are the functions and tasks that operate on the class data.

### Property Types

```systemverilog
class DataPacket;
    // Basic properties
    bit [7:0] id;
    int length;
    real timestamp;
    string source;
    
    // Array properties
    bit [7:0] data[];
    int status_flags[4];
    
    // Random properties
    rand bit [15:0] sequence_num;
    randc bit [3:0] priority;
    
    // Constraints on random properties
    constraint valid_priority {
        priority inside {[1:8]};
    }
    
    constraint data_size {
        length > 0;
        length < 1024;
        data.size() == length;
    }
endclass
```

### Method Types

```systemverilog
class NetworkPacket;
    bit [47:0] src_mac;
    bit [47:0] dst_mac;
    bit [15:0] ethertype;
    bit [7:0] payload[];
    
    // Constructor
    function new(bit [47:0] src = 0, bit [47:0] dst = 0);
        src_mac = src;
        dst_mac = dst;
        ethertype = 16'h0800; // IP
    endfunction
    
    // Function method (returns a value)
    function int get_payload_size();
        return payload.size();
    endfunction
    
    // Task method (can consume time)
    task send_packet();
        #10ns; // Simulate transmission delay
        $display("Packet sent from %h to %h", src_mac, dst_mac);
    endtask
    
    // Virtual method (can be overridden)
    virtual function void print_header();
        $display("SRC: %h, DST: %h, Type: %h", 
                 src_mac, dst_mac, ethertype);
    endfunction
    
    // Static method (belongs to class, not instance)
    static function bit [15:0] calculate_checksum(bit [7:0] data[]);
        bit [15:0] sum = 0;
        foreach(data[i]) sum += data[i];
        return ~sum;
    endfunction
endclass
```

### **Example 4: Data Packet Properties**
Class showcasing different types of properties: basic types, arrays, random variables with constraints.

In [35]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_4__data_packet_properties/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// data_packet_properties.sv
class DataPacket;
  // Basic properties
  bit [7:0]   packet_id;               // Packet identifier
  bit [15:0]  packet_length;           // Packet length
  string      packet_type;             // Packet type description
  
  // Array properties
  bit [7:0]   payload_data[];          // Dynamic array for payload
  bit [31:0]  header_fields[4];        // Fixed array for header fields
  
  // Additional properties
  bit [3:0]   priority_level;          // Packet priority (0-15)
  bit [7:0]   destination_addr;        // Destination address
  bit         valid_packet;            // Packet validity flag
  
  // Constructor
  function new();
    packet_type = "DATA";              // Default packet type
    valid_packet = 1'b1;               // Default to valid
    priority_level = 4'd3;             // Default priority
    destination_addr = 8'h42;          // Default destination
    packet_id = 8'h01;                 // Default ID
    packet_length = 16'd128;           // Default length
    
    // Initialize header fields
    foreach(header_fields[i]) begin
      header_fields[i] = 32'hDEADBEEF + i;
    end
    
    // Initialize payload with default size
    set_payload_size(108);             // 128 - 20 byte header
  endfunction
  
  // Method to set payload size and initialize with pattern
  function void set_payload_size(int size);
    payload_data = new[size];
    foreach(payload_data[i]) begin
      payload_data[i] = 8'(8'hA0 + (i % 16));
    end
  endfunction
  
  // Method to create a control packet
  function void make_control_packet();
    packet_type = "CONTROL";
    packet_length = 16'd64;
    priority_level = 4'd6;             // High priority
    packet_id = 8'($urandom() & 8'hFF);
    destination_addr = 8'h10;          // Control address
    set_payload_size(44);              // 64 - 20 byte header
  endfunction
  
  // Method to create a broadcast packet
  function void make_broadcast_packet();
    packet_type = "BROADCAST";
    packet_length = 16'd256;
    priority_level = 4'd7;             // Maximum priority
    packet_id = 8'($urandom() & 8'hFF);
    destination_addr = 8'hFE;          // Near-broadcast address
    set_payload_size(236);             // 256 - 20 byte header
  endfunction
  
  // Method to create a data packet with random properties
  function void make_data_packet();
    packet_type = "DATA";
    packet_length = 16'(64 + ($urandom() % 1455)); // 64-1518 range
    priority_level = 4'(1 + ($urandom() % 7));     // 1-7 range
    packet_id = 8'($urandom() & 8'hFF);
    destination_addr = 8'(1 + ($urandom() % 254)); // 1-254 range
    set_payload_size(int'(packet_length) - 20);
  endfunction
  
  // Method to display packet information
  function void display_packet();
    $display("=== Data Packet Information ===");
    $display("Packet ID: 0x%02h", packet_id);
    $display("Packet Type: %s", packet_type);
    $display("Packet Length: %0d bytes", packet_length);
    $display("Priority Level: %0d", priority_level);
    $display("Destination Address: 0x%02h", destination_addr);
    $display("Valid Packet: %s", valid_packet ? "YES" : "NO");
    $display("Payload Size: %0d bytes", payload_data.size());
    
    // Display header fields
    $display("Header Fields:");
    foreach(header_fields[i]) begin
      $display("  Field[%0d]: 0x%08h", i, header_fields[i]);
    end
    
    // Display first few payload bytes
    if (payload_data.size() > 0) begin
      $display("Payload Preview (first 8 bytes):");
      for (int i = 0; i < 8 && i < payload_data.size(); i++) begin
        $display("  payload_data[%0d]: 0x%02h", i, payload_data[i]);
      end
    end
    $display("===============================");
  endfunction
  
endclass
```

```systemverilog
// data_packet_properties_testbench.sv
module packet_testbench;
  
  // Create packet instances
  DataPacket data_pkt;
  DataPacket control_pkt;
  DataPacket broadcast_pkt;
  
  initial begin
    // Dump waves for simulation
    $dumpfile("packet_testbench.vcd");
    $dumpvars(0, packet_testbench);
    
    $display();
    
    // Test 1: Create and configure a standard data packet
    $display("--- Test 1: Standard Data Packet ---");
    data_pkt = new();
    data_pkt.make_data_packet();
    data_pkt.display_packet();
    $display();
    
    // Test 2: Create a control packet
    $display("--- Test 2: Control Packet ---");
    control_pkt = new();
    control_pkt.make_control_packet();
    control_pkt.display_packet();
    $display();
    
    // Test 3: Create a broadcast packet
    $display("--- Test 3: High Priority Broadcast Packet ---");
    broadcast_pkt = new();
    broadcast_pkt.make_broadcast_packet();
    broadcast_pkt.display_packet();
    $display();
    
    // Test 4: Demonstrate array property manipulation
    $display("--- Test 4: Array Property Manipulation ---");
    data_pkt.packet_type = "MODIFIED";
    
    // Manually modify some array elements
    data_pkt.header_fields[0] = 32'hCAFEBABE;
    data_pkt.header_fields[1] = 32'h12345678;
    
    // Modify payload with custom pattern
    data_pkt.set_payload_size(16);
    for (int i = 0; i < data_pkt.payload_data.size(); i++) begin
      data_pkt.payload_data[i] = 8'(8'hF0 + i);
    end
    data_pkt.packet_length = 16'd36; // 16 + 20 header
    
    data_pkt.display_packet();
    $display();
    
    // Test 5: Create multiple different packet types
    $display("--- Test 5: Multiple Packet Variations ---");
    for (int test_num = 1; test_num <= 3; test_num++) begin
      $display("Creating packet variation #%0d:", test_num);
      data_pkt.make_data_packet();
      $display(
        "  ID: 0x%02h, Length: %0d, Priority: %0d, Dest: 0x%02h",
        data_pkt.packet_id, data_pkt.packet_length, 
        data_pkt.priority_level, data_pkt.destination_addr);
    end
    $display();
    
    // Test 6: Demonstrate different packet sizes
    $display("--- Test 6: Different Packet Sizes ---");
    
    // Small packet
    data_pkt.packet_type = "SMALL";
    data_pkt.packet_length = 16'd64;
    data_pkt.set_payload_size(44);
    $display(
        "Small packet - Length: %0d, Payload: %0d bytes", 
        data_pkt.packet_length, data_pkt.payload_data.size());
    
    // Large packet
    data_pkt.packet_type = "LARGE";
    data_pkt.packet_length = 16'd1518;
    data_pkt.set_payload_size(1498);
    $display(
        "Large packet - Length: %0d, Payload: %0d bytes", 
        data_pkt.packet_length, data_pkt.payload_data.size());
    $display();
    
    $display("========================================");
    $display("Simulation completed successfully!");
    $display("All packet properties demonstrated:");
    $display("- Basic types (ID, length, type)");
    $display("- Arrays (fixed header, dynamic payload)");
    $display("- Property manipulation methods");
    $display("- Different packet configurations");
    $display("========================================");
    
    #10;  // Wait before finishing
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:

--- Test 1: Standard Data Packet ---
=== Data Packet Information ===
Packet ID: 0x02
Packet Type: DATA
Packet Length: 1469 bytes
Priority Level: 5
Destination Address: 0xfb
Valid Packet: YES
Payload Size: 1449 bytes
Header Fields:
  Field[0]: 0xdeadbeef
  Field[1]: 0xdeadbef0
  Field[2]: 0xdeadbef1
  Field[3]: 0xdeadbef2
Payload Preview (first 8 bytes):
  payload_data[0]: 0xa0
  payload_data[1]: 0xa1
  payload_data[2]: 0xa2
  payload_data[3]: 0xa3
  payload_data[4]: 0xa4
  payload_data[5]: 0xa5
  payload_data[6]: 0xa6
  payload_data[7]: 0xa7

--- Test 2: Control Packet ---
=== Data Packet Information ===
Packet ID: 0x78
Packet Type: CONTROL
Packet Length: 64 bytes
Priority Level: 6
Destination Address: 0x10
Valid Packet: YES
Payload Size: 44 bytes
Header Fields:
  Field[0]: 0xdeadbeef
  Field[1]: 0xdeadbef0
  Field[2]: 0xdeadbef1
  Field[3]: 0xdeadbef2
Payload Preview (first 8 bytes):
  payload_data[0]: 0xa0
  payload_data[1]: 0xa1
  payload_data[2]: 0xa2


0

### **Example 5: Network Packet Methods**
Demonstrates different method types: constructor, function, task, virtual, and static methods.

### **Example 6: Method Return Types**
Examples of methods that return values vs. methods that perform actions without returning data.

## Object Creation and Destruction

Objects are instances of classes created using the `new()` constructor. SystemVerilog handles memory management automatically.

### Object Creation

```systemverilog
class ConfigBlock;
    bit [31:0] base_addr;
    bit [7:0] version;
    bit enable;
    
    function new(bit [31:0] addr = 32'h1000);
        base_addr = addr;
        version = 8'h01;
        enable = 1'b1;
    endfunction
    
    function void configure(bit [31:0] addr, bit en);
        base_addr = addr;
        enable = en;
    endfunction
endclass

// Usage example
module test_objects;
    ConfigBlock cfg1, cfg2, cfg3;
    
    initial begin
        // Create objects
        cfg1 = new();                    // Use default constructor
        cfg2 = new(32'h2000);           // Pass parameter to constructor
        cfg3 = new(32'h3000);
        
        // Use objects
        cfg1.configure(32'h1500, 1'b0);
        cfg2.version = 8'h02;
        
        // Display object contents
        $display("Config 1: Addr=%h, Ver=%h, En=%b", 
                 cfg1.base_addr, cfg1.version, cfg1.enable);
        $display("Config 2: Addr=%h, Ver=%h, En=%b", 
                 cfg2.base_addr, cfg2.version, cfg2.enable);
    end
endmodule
```

### Object Assignment and Copying

```systemverilog
class DataBuffer;
    bit [7:0] buffer[];
    int size;
    
    function new(int sz = 16);
        size = sz;
        buffer = new[sz];
    endfunction
    
    // Deep copy method
    function DataBuffer copy();
        DataBuffer new_buf = new(size);
        new_buf.buffer = new[size];
        foreach(buffer[i]) new_buf.buffer[i] = buffer[i];
        return new_buf;
    endfunction
    
    function void fill_random();
        foreach(buffer[i]) buffer[i] = $random;
    endfunction
endclass

// Usage
module test_copy;
    DataBuffer buf1, buf2, buf3;
    
    initial begin
        buf1 = new(32);
        buf1.fill_random();
        
        buf2 = buf1;        // Shallow copy (both handles point to same object)
        buf3 = buf1.copy(); // Deep copy (creates new object)
        
        // Modify original
        buf1.buffer[0] = 8'hFF;
        
        // buf2 sees the change, buf3 doesn't
        $display("buf1[0] = %h", buf1.buffer[0]); // FF
        $display("buf2[0] = %h", buf2.buffer[0]); // FF (same object)
        $display("buf3[0] = %h", buf3.buffer[0]); // original value
    end
endmodule
```

### **Example 7: Configuration Block Objects**
Simple example of creating multiple objects with different constructor parameters.

### **Example 8: Object Assignment Demo**
Shows the difference between shallow copy (handle assignment) and deep copy methods.

### **Example 9: Buffer Management**
Demonstrates object creation, modification, and memory management concepts.

## The `this` Keyword

The `this` keyword refers to the current object instance and is used to resolve naming conflicts or for explicit reference.

```systemverilog
class Counter;
    int count;
    string name;
    
    function new(string name, int count = 0);
        this.name = name;    // Distinguish parameter from property
        this.count = count;  // Explicit reference to object property
    endfunction
    
    function void increment(int count = 1);
        this.count += count; // Use this to access object property
    endfunction
    
    function Counter get_copy();
        Counter copy = new(this.name, this.count);
        return copy;
    endfunction
    
    function void compare_with(Counter other);
        if (this.count > other.count)
            $display("%s (%0d) > %s (%0d)", 
                     this.name, this.count, other.name, other.count);
        else if (this.count < other.count)
            $display("%s (%0d) < %s (%0d)", 
                     this.name, this.count, other.name, other.count);
        else
            $display("%s and %s have equal counts (%0d)", 
                     this.name, other.name, this.count);
    endfunction
endclass

// Usage
module test_this;
    Counter c1, c2;
    
    initial begin
        c1 = new("Counter1", 5);
        c2 = new("Counter2", 3);
        
        c1.increment(2);
        c1.compare_with(c2);
        
        c2 = c1.get_copy();
        c2.name = "Counter2_copy";
        c1.compare_with(c2);
    end
endmodule
```

### **Example 10: Counter Class with this**
Class using `this` keyword to resolve naming conflicts between parameters and properties.

### **Example 11: Object Self-Reference**
Examples of methods that return references to the current object using `this`.

### **Example 11: Object Comparison**
Method that compares the current object with another object using `this`.

## Class Scope and Lifetime

Class scope defines the visibility of class members, while lifetime determines when objects are created and destroyed.

### Access Control

```systemverilog
class SecureData;
    // Public members (default)
    string public_info;
    
    // Protected members (accessible in derived classes)
    protected bit [31:0] protected_key;
    
    // Local members (private to this class)
    local bit [127:0] private_data;
    local bit [7:0] secret_code;
    
    function new(string info = "default");
        public_info = info;
        protected_key = $random;
        private_data = {$random, $random, $random, $random};
        secret_code = 8'hA5;
    endfunction
    
    // Public method to access private data
    function bit [31:0] get_hash();
        return private_data[31:0] ^ protected_key ^ {24'b0, secret_code};
    endfunction
    
    // Protected method for derived classes
    protected function bit [31:0] get_protected_key();
        return protected_key;
    endfunction
    
    // Local method (private)
    local function bit verify_secret(bit [7:0] code);
        return (code == secret_code);
    endfunction
endclass

// Extended class demonstrating scope
class ExtendedSecureData extends SecureData;
    function new(string info = "extended");
        super.new(info);
    endfunction
    
    function void show_protected();
        // Can access protected members
        $display("Protected key: %h", protected_key);
        $display("Using protected method: %h", get_protected_key());
        
        // Cannot access local/private members
        // $display("Secret: %h", secret_code); // Error!
    endfunction
endclass
```

### Object Lifetime

```systemverilog
class Resource;
    static int instance_count = 0;
    int id;
    string name;
    
    function new(string name);
        instance_count++;
        id = instance_count;
        this.name = name;
        $display("Resource %0d (%s) created", id, name);
    endfunction
    
    // Destructor-like method (called explicitly)
    function void cleanup();
        $display("Resource %0d (%s) cleaned up", id, name);
        // Custom cleanup code here
    endfunction
    
    static function int get_instance_count();
        return instance_count;
    endfunction
endclass

module test_lifetime;
    Resource res1, res2;
    
    initial begin
        $display("Initial count: %0d", Resource::get_instance_count());
        
        res1 = new("Resource1");
        res2 = new("Resource2");
        
        $display("After creation: %0d", Resource::get_instance_count());
        
        // Objects are automatically garbage collected when no longer referenced
        res1 = null; // Remove reference
        
        // Explicit cleanup (if needed)
        res2.cleanup();
        res2 = null;
        
        // Note: instance_count doesn't decrease (no automatic destructor)
        $display("Final count: %0d", Resource::get_instance_count());
    end
endmodule
```

### **Example 12: Access Control Demo**
Class with public, protected, and local (private) members showing visibility rules.

### **Example 13: Secure Data Class**
Demonstrates encapsulation with different access levels and methods to access private data.

### **Example 14: Resource Tracking**
Class that tracks object creation and cleanup with instance counting.

## Static Members

Static members belong to the class rather than to individual instances. They are shared among all objects of the class.

### Static Properties and Methods

```systemverilog
class IDGenerator;
    static int next_id = 1;
    static int total_objects = 0;
    static string class_version = "v1.0";
    
    int object_id;
    string name;
    
    function new(string name);
        this.name = name;
        this.object_id = next_id++;
        total_objects++;
        $display("Created object %0d: %s", object_id, name);
    endfunction
    
    // Static method - can be called without creating an object
    static function int get_next_id();
        return next_id;
    endfunction
    
    static function int get_total_objects();
        return total_objects;
    endfunction
    
    static function void reset_counter();
        next_id = 1;
        total_objects = 0;
        $display("ID counter reset");
    endfunction
    
    // Static method to get class information
    static function string get_class_info();
        return $sformatf("IDGenerator %s - Next ID: %0d, Total: %0d",
                        class_version, next_id, total_objects);
    endfunction
    
    // Instance method that uses static data
    function void show_info();
        $display("Object %0d (%s) - Class has %0d total objects",
                object_id, name, total_objects);
    endfunction
endclass

// Usage of static members
module test_static;
    IDGenerator obj1, obj2, obj3;
    
    initial begin
        // Call static method without creating objects
        $display("Class info: %s", IDGenerator::get_class_info());
        $display("Next ID will be: %0d", IDGenerator::get_next_id());
        
        // Create objects
        obj1 = new("First");
        obj2 = new("Second");
        obj3 = new("Third");
        
        // Show object info
        obj1.show_info();
        obj2.show_info();
        
        // Access static members through class name
        $display("Total objects created: %0d", IDGenerator::get_total_objects());
        
        // Reset static data
        IDGenerator::reset_counter();
        
        // Create new object after reset
        obj1 = new("After Reset");
        $display("Final class info: %s", IDGenerator::get_class_info());
    end
endmodule
```

### Static vs Instance Members

```systemverilog
class BankAccount;
    static real interest_rate = 0.05;  // Static - same for all accounts
    static int total_accounts = 0;     // Static - count of all accounts
    
    int account_number;                // Instance - unique per account
    real balance;                      // Instance - individual balance
    string owner_name;                 // Instance - individual owner
    
    function new(string name, real initial_balance = 0.0);
        total_accounts++;
        account_number = total_accounts;
        owner_name = name;
        balance = initial_balance;
    endfunction
    
    // Static method to change interest rate for all accounts
    static function void set_interest_rate(real new_rate);
        interest_rate = new_rate;
        $display("Interest rate changed to %.2f%% for all accounts", 
                 new_rate * 100);
    endfunction
    
    // Instance method that uses both static and instance data
    function void apply_interest();
        real interest = balance * interest_rate;
        balance += interest;
        $display("Account %0d (%s): Interest $%.2f applied, new balance $%.2f",
                account_number, owner_name, interest, balance);
    endfunction
    
    // Instance method
    function void deposit(real amount);
        balance += amount;
        $display("Account %0d: Deposited $%.2f, balance now $%.2f",
                account_number, amount, balance);
    endfunction
    
    static function void print_statistics();
        $display("Bank Statistics:");
        $display("  Total accounts: %0d", total_accounts);
        $display("  Current interest rate: %.2f%%", interest_rate * 100);
    endfunction
endclass

module test_static_vs_instance;
    BankAccount acc1, acc2, acc3;
    
    initial begin
        // Create accounts
        acc1 = new("Alice", 1000.0);
        acc2 = new("Bob", 500.0);
        acc3 = new("Charlie", 1500.0);
        
        // Show initial statistics
        BankAccount::print_statistics();
        
        // Apply interest with current rate
        acc1.apply_interest();
        acc2.apply_interest();
        acc3.apply_interest();
        
        // Change interest rate (affects all accounts)
        BankAccount::set_interest_rate(0.08);
        
        // Apply new interest rate
        acc1.apply_interest();
        acc2.apply_interest();
        acc3.apply_interest();
        
        // Final statistics
        BankAccount::print_statistics();
    end
endmodule
```

### **Example 15: ID Generator Class**
Class using static properties to generate unique IDs and track total object count.

### **Example 16: Static Utility Methods**
Examples of static methods that can be called without creating class instances.

### **Example 17: Bank Account System**
Demonstrates the difference between static (shared) and instance (individual) data members.

### Best Practices

1. **Use constructors** to initialize object state properly
2. **Implement deep copy methods** when objects contain dynamic arrays or other objects
3. **Use `this` keyword** to resolve naming conflicts and improve code clarity
4. **Apply appropriate access control** (public, protected, local) to encapsulate data
5. **Use static members** for class-wide data and utility functions
6. **Implement cleanup methods** for resources that need explicit cleanup
7. **Design classes with single responsibility** for better maintainability

### Summary

Classes and objects in SystemVerilog provide powerful abstraction mechanisms for creating reusable and maintainable code. Key concepts include:

- **Class declarations** define templates for objects with properties and methods
- **Object creation** uses constructors and the `new()` operator
- **The `this` keyword** provides explicit reference to the current object
- **Class scope** controls member visibility and access
- **Static members** are shared across all instances of a class
- **Proper encapsulation** and access control improve code reliability

Understanding these concepts is essential for effective object-oriented programming in SystemVerilog, particularly for complex verification environments and testbenches.

# SystemVerilog Classes and Objects - Simple Examples

Based on Chapter 9: Classes and Objects, here are super simple examples organized by subchapter:

## Best Practices Examples

### **Constructor Best Practices**
Proper initialization of object state with parameter validation and default values.

### **Deep Copy Implementation**
How to implement proper deep copy methods for objects containing dynamic arrays.

### **Encapsulation Example**
Well-designed class showing proper use of access control and getter/setter methods.

### **Resource Management**
Class design patterns for objects that need explicit cleanup or resource management.

### **Single Responsibility**
Example of a class designed with a single, well-defined purpose for better maintainability.

### **Static vs Instance Usage**
Clear examples showing when to use static members vs instance members appropriately.